In [1]:

############################################################################
################## Importing Required Pacakges##############################
############################################################################

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from wordcloud import WordCloud, STOPWORDS
import numpy as np
import re
import os
import nltk

""" This block is for EDA on the given data"""

####################Importing csv file with names and authors################

#Assuming File location is local "Need to change it"

#Getting data into dataframe

%time
data = pd.read_csv("master996.csv", delimiter=';',encoding= 'unicode_escape')

#Checking memory usage in detail with distribution of data
data.info(memory_usage="deep")

#Checking the first 5 lines of the file

print(data.head(5))

#Checking the shape of the dataframe

data.shape

#Checking the count of the values in dataframe, should match wiht the file

data.count()

#Extracting the bookid into Dataframe column FileNo, book id pg10067- FileNo - 10067

data['FileNo'] = data['book_id'].str.replace(r'\D+', '').astype(int)

#Checkign the values

data.head(5)

#Checking Null Values

data.info() 

#There are no null values

#Getting list of gernes

print(data.guten_genre.value_counts().unique())

#Checking distribution of Gerne columns

data.guten_genre.describe()

#Getting distribution of authors

data.Author_Name.describe()

""" List is quite long"""

#Getting list of authors

# print(data.Author_Name.unique())


##################################### Trying some plots on the file ######################################

plt.figure(figsize=(18,5))

#See how much the Gernes are distributed
sns.countplot(data['guten_genre'])
plt.show()

"""Clearly Classes are imbalanced, Literarcy genre has higest values and Allegories almost have no books"""

#chekcing the actualy counts of each gerne

data.guten_genre.value_counts()


"""
Literary                       794
Detective and Mystery          111
Sea and Adventure               36
Love and Romance                18
Western Stories                 18
Humorous and Wit and Satire      6
Ghost and Horror                 6
Christmas Stories                5
Allegories                       2

"""

### Wordcloud to see most frequent author ###

stopwords = set(STOPWORDS)
wordcloud = WordCloud(stopwords=stopwords, background_color="black").generate(str(data['Author_Name']))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.show()

"""Looks like we have most fictions from Charles dickson and Stevenson"""


##################################  Here add more analysis and graphs for more insigths #####################
#############################################################################################################



# Add here number of books per author

# Add here number of gerne per author



################################################################################################################
##################################Importing html content into pnadas and joining it wiht meta data##############



Wall time: 0 ns
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 996 entries, 0 to 995
Data columns (total 4 columns):
Book_Name      996 non-null object
book_id        996 non-null object
guten_genre    996 non-null object
Author_Name    996 non-null object
dtypes: object(4)
memory usage: 287.5 KB
                                           Book_Name       book_id  \
0  The Mystery of the Boule Cabinet: A Detective ...  pg10067.epub   
1                                          The Pupil   pg1032.epub   
2                                     At Love's Cost  pg10379.epub   
3                             The Heart of the Range  pg10473.epub   
4                        The Worshipper of the Image  pg10812.epub   

             guten_genre               Author_Name  
0  Detective and Mystery  Stevenson| Burton Egbert  
1               Literary              James| Henry  
2               Literary          Garvice| Charles  
3        Western Stories  White| William Patterson  
4            

<Figure size 1800x500 with 1 Axes>

<Figure size 640x480 with 1 Axes>

'Looks like we have most fictions from Charles dickson and Stevenson'

In [2]:
import glob

# Getting all files and its names into dict and then converting to dataframe"""

Content = {}

#Its placed in Books folder at local , need to change it later 
%timeit
files = glob.glob("Books\*.html")
for f in files:
    with open(f,mode ='r', encoding = 'utf8') as myfile:
        Content[f]=myfile.read().replace("<br>", '\n')

#Can check for few files from below code        
"""for i in data:
    print(i, data[i])
"""
#Putting it in dataframe
df = pd.DataFrame.from_dict(Content,orient='index').reset_index()
#Renaming column names
df.columns = ['File_Name', 'Data']
#Adding FilNo column to join
df['FileNo'] = df['File_Name'].str.replace(r'\D+', '').astype(int)

df.info(memory_usage="deep")

left = df
right = data

#merging by matching the bookid, ignoring which files has not match
result = pd.merge(left, right , how='inner', on=['FileNo', 'FileNo'])

#Checking resulting dataframe
result.info()

#Dropping unnecessery columns

books=result.drop(['File_Name', 'book_id'], axis=1)
books.head(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 152 entries, 0 to 151
Data columns (total 3 columns):
File_Name    152 non-null object
Data         152 non-null object
FileNo       152 non-null int32
dtypes: int32(1), object(2)
memory usage: 78.6 MB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 152 entries, 0 to 151
Data columns (total 7 columns):
File_Name      152 non-null object
Data           152 non-null object
FileNo         152 non-null int32
Book_Name      152 non-null object
book_id        152 non-null object
guten_genre    152 non-null object
Author_Name    152 non-null object
dtypes: int32(1), object(6)
memory usage: 8.9+ KB


,Data,FileNo,Book_Name,guten_genre,Author_Name
0,<p>The shabby stranger seated himself familiar...,41089,The Sign of the Stranger,Detective and Mystery,Queux| William Le
1,"<p>Samson's Mill Settlement had, for the past ...",41122,Rayton: A Backwoods Mystery,Literary,Roberts| Theodore Goodridge
2,<p>He and she stood in a room in an inn in the...,41191,Jenifer's Prayer,Literary,Crane| Oliver
3,"<p>It was upon a dark and lowering afternoon, ...",41212,The Eve of All-Hallows- Adelaide of Tyrconnel-...,Literary,Hartstonge| Matthew Weld
4,"<p>The great game at chess, between Mr. Gammon...",41247,Ten Thousand a-Year. Volume 3.,Literary,Warren| Samuel


In [3]:
##################################### Pre-Processing of the html file content########################################

from sklearn import preprocessing
from bs4 import BeautifulSoup
import string

# labeling the classes

le = preprocessing.LabelEncoder()
books['guten_genre'] = le.fit_transform(books['guten_genre'])

#checking raw content
books['Data'].head(5)

#cleaning it with soup
books['Data']= [BeautifulSoup(text).get_text() for text in books['Data'] ]

#Make everything in lowe case
books['Data'] = books['Data'].apply(lambda x: x.lower())

#Remove puchtuation
books['Data'] = books['Data'].apply(lambda x: x.translate(str.maketrans('','', string.punctuation)))

#Remove Digits
books['Data'] = books['Data'].apply(lambda x: x.translate(str.maketrans('','', string.digits)))
books['Data'].str.strip()

books['Data'].head(5)

0    the shabby stranger seated himself familiarly ...
1    samsons mill settlement had for the past fifte...
2    he and she stood in a room in an inn in the to...
3    it was upon a dark and lowering afternoon the ...
4    the great game at chess between mr gammon and ...
Name: Data, dtype: object

In [4]:
#######################################BAG OF WORDS##########################################################

In [5]:
#import the necessary libraries
from sklearn.feature_extraction.text import CountVectorizer

# df_min= 0.and df_max = 1. the value could be its frequency in the document, 
#occurrence (denoted by 1 or 0) or even weighted values.

cv_vect = CountVectorizer(min_df=0., max_df=1.)

cv_matrix = cv_vect.fit_transform(books['Data'])

cv_matrix


<152x91656 sparse matrix of type '<class 'numpy.int64'>'
	with 984208 stored elements in Compressed Sparse Row format>

In [6]:
#To view thw result in the matrix form
cv_matrix = cv_matrix.toarray()
cv_matrix

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [7]:
# get all unique words in the corpus

vocab1 = cv_vect.get_feature_names()

# show document feature vectors

pd.DataFrame(cv_matrix, columns=vocab1)

,aa,aaaamen,aaalready,aacannibal,aah,aamen,aao,aarguin,aarons,aaskin,...,ñandú,ñandús,ño,ôtezmoi,œdipus,œil,βοῦς,δδ,κέφαλος,μήδεια
0,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,1,0,1,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
147,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
148,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
149,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
150,0,1,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
#########################################BAG OF N-GRAM MODEL##########################################
#N gram model is implemented in order to get the words that occur in sequence. In case of BOW the order of
# the words is not considered.So inorder to extract phrases or collection of words which occur in a sequence we use N gram
#The below code depicts bigram model, n=2

**FOR SIMPLICITY I AM USING ONLY 69 Documents out of 996. When you are executing the code there is no need to change anything except the size used which has been indicated whereever it is**

In [10]:
# you can set the n-gram range to 1,2 to get unigrams as well as bigrams

bv = CountVectorizer(ngram_range=(1,2))

start = 0
while start < len(books['Data']):
    bv.fit(books['Data'][start:(start+20)]) #when executing all the 996 docs change the batch size to 300 or more according to your convinience
    start += 20 #same change is applied inorder to iterate
bv_matrix = bv.fit_transform(books['Data'])
bv_matrix = bv_matrix.toarray()

vocab2 = bv.get_feature_names()

pd.DataFrame(bv_matrix, columns = vocab2)

#bv = CountVectorizer(ngram_range=(1,2))

#bv_matrix = bv.fit_transform(books['Data'])

#bv_matrix = bv_matrix.toarray()

#vocab2 = bv.get_feature_names()

#pd.DataFrame(bv_matrix, columns=vocab2)

,aa,aa fool,aa shilling,aa visitor,aaaamen,aaaamen an,aaalready,aaalready married,aacannibal,aacannibal said,...,œil,œil bel,βοῦς,βοῦς κέφαλος,δδ,δδ hand,κέφαλος,κέφαλος student,μήδεια,μήδεια anglicé
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,1,1,1,1,0,0,1,1,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
147,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
148,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
149,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
150,0,0,0,0,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
# for Author name and set it to trigram #you can make out the difference
bv = CountVectorizer(ngram_range=(3,3))

start = 0
while start < len(books['Data']):
    bv.fit(books['Data'][start:(start+20)]) #when executing all the 996 docs change the batch size to 300 or more according to your convinience
    start += 20 #same change is applied inorder to iterate
bv_matrix = bv.fit_transform(books['Data'])
bv_matrix = bv_matrix.toarray()

vocab2 = bv.get_feature_names()

pd.DataFrame(bv_matrix, columns = vocab2)

#bv = CountVectorizer(ngram_range=(3,3))

#bv_matrix = bv.fit_transform(books['Data'])

#bv_matrix = bv_matrix.toarray()

#vocab2 = bv.get_feature_names()

#pd.DataFrame(bv_matrix, columns=vocab2)

In [11]:
#######################TF IDF MODEL###########################################
#tfidf(w, D) is the TF-IDF score for word w in document D. The term tf(w, D) represents the term frequency of the word w 
#in document D, which can be obtained from the Bag of Words model. 
#The term idf(w, D) is the inverse document frequency for the term w

from sklearn.feature_extraction.text import TfidfVectorizer

tv = TfidfVectorizer(min_df=0., max_df=1., use_idf=True)

tv_matrix = tv.fit_transform(books['Data'])

tv_matrix = tv_matrix.toarray()


vocab3 = tv.get_feature_names()

pd.DataFrame((tv_matrix), columns=vocab3)

,aa,aaaamen,aaalready,aacannibal,aah,aamen,aao,aarguin,aarons,aaskin,...,ñandú,ñandús,ño,ôtezmoi,œdipus,œil,βοῦς,δδ,κέφαλος,μήδεια
0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000729,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0
1,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0
2,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0
3,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.001139,0.001139,0.000000,0.001139,0.0
4,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000356,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
147,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0
148,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0
149,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0
150,0.0,0.000704,0.0,0.0,0.0,0.000704,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0


In [12]:
#cosine similarity and compare pairwise document similarity based on their TF-IDF feature vectors
#####################Document Similarity###############################################

In [13]:
from sklearn.metrics.pairwise import cosine_similarity

similarity_matrix = cosine_similarity(tv_matrix)

similarity_df = pd.DataFrame(similarity_matrix)

similarity_df

,0,1,2,3,4,5,6,7,8,9,...,142,143,144,145,146,147,148,149,150,151
0,1.000000,0.828803,0.839751,0.897193,0.856997,0.904715,0.928450,0.900465,0.903563,0.852882,...,0.875456,0.901017,0.894800,0.892845,0.900869,0.928604,0.895165,0.875512,0.734219,0.851474
1,0.828803,1.000000,0.763097,0.811336,0.808811,0.843609,0.823101,0.831250,0.822207,0.763145,...,0.825128,0.851561,0.815250,0.821175,0.824558,0.837575,0.795542,0.798972,0.711761,0.753783
2,0.839751,0.763097,1.000000,0.788101,0.789016,0.810422,0.813445,0.803955,0.801039,0.838450,...,0.793545,0.812202,0.787076,0.799108,0.799158,0.841015,0.809003,0.800973,0.671592,0.789197
3,0.897193,0.811336,0.788101,1.000000,0.876662,0.909268,0.935245,0.886817,0.901628,0.775306,...,0.862779,0.893376,0.879809,0.858367,0.876204,0.878290,0.816692,0.845285,0.695670,0.794514
4,0.856997,0.808811,0.789016,0.876662,1.000000,0.869180,0.867010,0.853622,0.856494,0.787747,...,0.850879,0.867800,0.840417,0.839024,0.846334,0.863419,0.812932,0.828088,0.693971,0.787406
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
147,0.928604,0.837575,0.841015,0.878290,0.863419,0.899203,0.916539,0.899474,0.897691,0.857466,...,0.881772,0.902727,0.908159,0.910291,0.916337,1.000000,0.919286,0.866750,0.744600,0.841286
148,0.895165,0.795542,0.809003,0.816692,0.812932,0.850704,0.871609,0.857335,0.852942,0.831304,...,0.841252,0.860352,0.873301,0.879554,0.883226,0.919286,1.000000,0.829475,0.713908,0.812826
149,0.875512,0.798972,0.800973,0.845285,0.828088,0.860521,0.869108,0.860734,0.859824,0.832394,...,0.828984,0.860651,0.840445,0.842766,0.848184,0.866750,0.829475,1.000000,0.718244,0.942071
150,0.734219,0.711761,0.671592,0.695670,0.693971,0.725370,0.722339,0.726911,0.722682,0.681925,...,0.717955,0.737861,0.717000,0.722590,0.725372,0.744600,0.713908,0.718244,1.000000,0.675283


In [14]:
##############################LDA MODEL#######################
#when LDA is applied the document term matrix gets decomposed into document term matrix and topic term matrix
####to get the document-topic matrix 

In [15]:
from sklearn.decomposition import LatentDirichletAllocation

lda = LatentDirichletAllocation(n_components=3, max_iter=50, random_state=0, batch_size = 30) #change the max_iter to 1000 or more and also 
#Batch_size to 300 or more

dt_matrix = lda.fit_transform(cv_matrix)

features = pd.DataFrame(dt_matrix, columns=['T1', 'T2', 'T3'])

features

,T1,T2,T3
0,0.174353,0.710703,0.114944
1,0.000038,0.999902,0.000060
2,0.010733,0.008349,0.980918
3,0.997057,0.000010,0.002933
4,0.913991,0.065744,0.020265
...,...,...,...
147,0.056823,0.829764,0.113414
148,0.053804,0.649602,0.296594
149,0.160534,0.003915,0.835551
150,0.000005,0.999989,0.000006


In [16]:
tt_matrix = lda.components_

for topic_weights in tt_matrix:

    topic = [(token, weight) for token, weight in zip(vocab3, topic_weights)]

    topic = sorted(topic, key=lambda x: -x[1])

    topic = [item for item in topic if item[1] > 0.6]

    #print(topic)

    #print()

In [17]:
################################FEATURE SCALING##################################
from sklearn import decomposition
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler

# to handle values with varying magnitude 
fs_f = data.iloc[:, 1:3].values 

print ("\nOriginal data values : \n",  fs) 
  
  

""""Handling the missing values """
  
from sklearn import preprocessing 
  
""" MIN MAX SCALER """
  
min_max_scaler = preprocessing.MinMaxScaler(feature_range =(0, 1)) 
  
# Scaled feature 
fs_after_min_max_scaler = min_max_scaler.fit_transform(fs) 
  
print ("\nAfter min max Scaling : \n", fs_after_min_max_scaler) 
  
  
""" Standardisation """
  
Standardisation = preprocessing.StandardScaler() 
  
# Scaled feature 
fs_after_Standardisation = Standardisation.fit_transform(fs) 
  
print ("\nAfter Standardisation : \n", fs_after_Standardisation) 

In [18]:
# Standardize Features
sc = StandardScaler(with_mean = True, with_std = True)

# Fit the scaler to the features and transform
fe_std = sc.fit_transform(cv_matrix)

# View the new feature data's shape    
print(); print(fe_std.shape)
print(); print(fe_std)



(152, 91656)

[[-0.13363062 -0.08137885 -0.08137885 ... -0.08137885 -0.08137885
  -0.08137885]
 [-0.13363062 -0.08137885 -0.08137885 ... -0.08137885 -0.08137885
  -0.08137885]
 [-0.13363062 -0.08137885 -0.08137885 ... -0.08137885 -0.08137885
  -0.08137885]
 ...
 [-0.13363062 -0.08137885 -0.08137885 ... -0.08137885 -0.08137885
  -0.08137885]
 [-0.13363062 12.28820573 -0.08137885 ... -0.08137885 -0.08137885
  -0.08137885]
 [-0.13363062 -0.08137885 -0.08137885 ... -0.08137885 -0.08137885
  -0.08137885]]


In [19]:
# Create a pca object with the 3 components
pca = decomposition.PCA(n_components=3)

# Fit the PCA and transform the data
fe_std_pca = pca.fit_transform(fe_std)

# View the new feature data's shape
print(); print(fe_std_pca.shape)
print(); print(fe_std_pca)


(152, 3)

[[-5.78676133e+00 -1.38602162e+00 -9.04042115e-02]
 [-2.53801172e+01  3.39822734e-01  5.27728552e+00]
 [-3.96171441e+01  5.16676791e+00  2.32084168e+01]
 [-1.70670657e+00  1.21317346e+01  3.23679709e+01]
 [ 1.05044362e+02 -2.39265654e+01 -7.76404799e+01]
 [-3.66612693e+01  5.37884507e+00  1.98893788e+01]
 [-2.65072504e+01  2.53681994e+00  1.32958459e+01]
 [-2.71435065e+01  2.12403052e+00  1.23853416e+01]
 [-2.84374926e+01  2.83451924e+00  1.48040179e+01]
 [-2.03622922e+01  1.15931817e+00  8.32128340e+00]
 [-2.50854619e+01  1.95657089e+00  1.15345419e+01]
 [-2.40643609e+01  1.70148842e+00  1.03716455e+01]
 [ 7.85830900e+01 -1.63766535e+01 -5.43895037e+01]
 [-1.63892248e+01  4.08949317e+00  5.89420058e+00]
 [-1.96922624e+01  2.79075533e+00  6.98304297e+00]
 [-1.57029114e+01  4.06477771e+00  5.51159410e+00]
 [-1.40185072e+00  3.85159626e+00  1.06409186e+00]
 [-7.91673353e+00  2.76064341e+00  5.44498704e+00]
 [-8.98895465e+00  2.74335040e+00  3.79452095e+00]
 [-1.10082405e+01 -6

**Feature Selection**

**CHI SQUARE**

**MUTUAL INFORMATION**

when to use chi2?

When the data type of our feature to be tested and the target variable are both categorical (i.e. we have a classification problem) we can use Chi-Squared test.

In [20]:
####################CHI SQUARE######################### Need of verification if the implementation is correct
from sklearn.feature_selection import chi2, SelectKBest
chi2score = chi2(cv_matrix, books['Data'])
chi2score


(array([224., 151., 151., ..., 151., 151., 151.]),
 array([1.05886377e-04, 4.84694541e-01, 4.84694541e-01, ...,
        4.84694541e-01, 4.84694541e-01, 4.84694541e-01]))

In [21]:
#####SELECT K BEST#######
kbest = SelectKBest(score_func = chi2, k = 30)
cv_kbest = kbest.fit_transform(cv_matrix, books['Data']) #check the parameters passed is correct
cv_kbest

array([[   0, 2233,    0, ...,    0,    0,    0],
       [   0, 2009,    0, ...,    0,    0,    0],
       [   0, 1168,    0, ...,    0,    0,    0],
       ...,
       [   0, 1436,    0, ...,    0,    0,    0],
       [ 734, 1673,    0, ...,    0,    0,    0],
       [   0, 1840,    0, ...,    0,    0,    0]], dtype=int64)

In [22]:
X_new = SelectKBest(chi2, k=20).fit_transform(cv_matrix, books['Data'])
X_new.shape

(152, 20)

In [24]:
############################Mutual Information#################
from sklearn.feature_selection import mutual_info_classif
res = dict(zip(vocab1, mutual_info_classif(cv_matrix, books['Data'], discrete_features=True)))
#print(res)

**FEATURE REPRESENTATION**

**WORD2VEC**

**GLOVE**

**FAST TEXT**